In [1]:

%load_ext autoreload
%autoreload 2

from module_test.raw_code.DataManagers.DataManagers import (
    BulkOptionQueryRequestParameter,
    OptionQueryRequestParameter,
    ChainDataRequest,
    save_to_database,
    save_chain_data,
    _ManagerLazyLoader,
    _SaveManager
)
import pandas as pd
from datetime import datetime
import os, json, sys
import numpy as np
from trade.helpers.Logging import setup_logger


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-04-29 22:25:47 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs
Using Proxy URL: http://18.232.166.224:5500/thetadata


In [19]:
import json
with open('requests.jsonl', 'r') as f:
    items = [json.loads(x) for x in f.readlines()]

for item in items:
    item['save_func'] = eval(item['save_func'])

_SaveManager.enqueue(item)

[ProcessSaveManager] Auto-setup completed.
[SaveManager] Enqueueing save request for AAPL on {'exp': '2025-09-19', 'right': 'C', 'strike': 13.0, 'start': '2024-10-23', 'end': '2025-04-27', 'tick': 'AAPL', 'type_': 'bulk', 'save_func': functools.partial(<function save_to_database at 0x140397560>, pool=False), '_requests': <ListProxy object, typeid 'list' at 0x1402ebe10>}


Saving to cache from db
Updating db rates data
Saving to cache from dbs_timeseries: 7
[ProcessSaveManager] Current requests: {'SaveWorker-0': <module_test.raw_code.DataManagers.Requests.BulkOptionQueryRequestParameter object at 0x140a40990>}, inside lock
[ProcessSaveManager] Processing save request for AAPL on <module_test.raw_code.DataManagers.Requests.BulkOptionQueryRequestParameter object at 0x13f4881d0>, thread SaveWorker-0
Worker SaveWorker-0 got a request
Worker SaveWorker-0 got a request: <module_test.raw_code.DataManagers.Requests.BulkOptionQueryRequestParameter object at 0x13f4881d0>
2025-04-29 21:27:54 trade.helpers.helper ERROR: Binomial Implied vol took too long to calculate for 228.02000427246094, 300.0, 0.04402999877929688, 71.975, 2025-09-19 00:00:00, p
2025-04-29 21:29:24 trade.helpers.helper ERROR: Binomial Implied vol took too long to calculate for 236.8699951171875, 300.0, 0.04228000164031982, 63.125, 2025-09-19 00:00:00, p
2025-04-29 21:48:12 trade.helpers.helper ER

In [23]:
# items = [1,2,3]
items.pop(0), items

(3, [])

In [6]:
item

{'exp': '2024-09-17',
 'start': '2024-09-17',
 'end': '2024-09-17',
 'tick': 'JPM',
 'save_func': <function module_test.raw_code.DataManagers.DataManagers.save_chain_data(data_request, **kwargs)>,
 'set_attributes': {'post_processed_data':                build_date ticker           expiration  strike right  bid_size  \
  0     2024-09-17 00:00:00    JPM  2024-10-18 00:00:00   220.0     P        42   
  1     2024-09-17 00:00:00    JPM  2024-10-18 00:00:00   220.0     C        10   
  2     2024-09-17 00:00:00    JPM  2024-10-25 00:00:00   220.0     P        22   
  3     2024-09-17 00:00:00    JPM  2024-10-25 00:00:00   220.0     C         7   
  4     2024-09-17 00:00:00    JPM  2024-11-01 00:00:00   220.0     C        40   
  ...                   ...    ...                  ...     ...   ...       ...   
  1721  2024-09-17 00:00:00    JPM  2025-03-21 00:00:00   215.0     P        10   
  1722  2024-09-17 00:00:00    JPM  2024-09-20 00:00:00   220.0     P        46   
  1723  2024-09

In [22]:
_SaveManager.status()

2025-04-29 20:01:19 INFO: Backing off send_request(...) for 0.8s (requests.exceptions.ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))
Saving to cache from db


{'pending_tasks': 0,
 'max_queue_size': 100,
 'active_processes': 4,
 'total_processes': 4,
 'current_requests': {'SaveWorker-0': <module_test.raw_code.DataManagers.Requests.BulkOptionQueryRequestParameter at 0x140a1b950>},
 'num_finished_requests': 0,
 'num_failed_requests': 0,
 'failed_initialization': 0}

In [10]:
eval('save_to_database')

<function module_test.raw_code.DataManagers.DataManagers.save_to_database(data_request: module_test.raw_code.DataManagers.Requests.OptionQueryRequestParameter, print_info: bool = False, pool: bool = True, **kwargs) -> None>

## SAVE OBJECTS AS DICT

In [2]:
objs = {
    'BulkOptionQueryRequestParameter': BulkOptionQueryRequestParameter,
    'OptionQueryRequestParameter': OptionQueryRequestParameter,
    'ChainDataRequest': ChainDataRequest,
}

## LOAD ERRORS

In [3]:
from queue import Queue
error_queue = Queue()

In [4]:
with open(f"{os.environ['WORK_DIR']}/module_test/raw_code/DataManagers/failed_request.jsonl", "r") as f:
    for line in f:
        error_queue.put(json.loads(line))

In [118]:
request_info = error_queue.get(block=False)

In [119]:
request_info["class_name"]

TypeError: string indices must be integers, not 'str'

In [110]:
request = objs[request_info['class_name']](**request_info)

for key, value in request_info.items():
    if key not in ['class_name', 'request_id']:
        setattr(request, key, value)

lazy_loader = _ManagerLazyLoader(request.symbol)
request.input_params = getattr(lazy_loader, request.agg)
lazy_loader.exp = request.exp

In [111]:
request.error, request.symbol, request.agg, request.exp

('cannot reshape array of size 0 into shape (0,newaxis)',
 'SBUX',
 'eod',
 '2024-03-22 00:00:00')

In [71]:
save_to_database(request)

2025-04-24 00:06:22 trade.models.VolSurface ERROR: Expiration date is the same as the datetime. Returning 0
2025-04-24 00:06:30 trade.models.VolSurface ERROR: Expiration date is the same as the datetime. Returning 0
2025-04-24 00:06:52 trade.models.utils ERROR: AAPL20250321C40, 2024-09-25 could not fit SVI model. Returning zero
2025-04-24 00:09:29 trade.models.utils ERROR: AAPL20250321C40, 2025-01-30 could not fit SVI model. Returning zero
2025-04-24 00:10:50 trade.models.utils ERROR: AAPL20250321C40, 2025-03-17 has a poorly fitted model. Returning zero
2025-04-24 00:11:09 trade.models.VolSurface ERROR: Expiration date is the same as the datetime. Returning 0
2025-04-24 00:11:18 trade.models.VolSurface ERROR: Expiration date is the same as the datetime. Returning 0
2025-04-24 00:11:23 DataManager.py ERROR: 
2025-04-24 00:11:23 DataManager.py ERROR: resolve_missing_vols_in_data raise an error: positional indexers are out-of-bounds
Traceback (most recent call last):
  File "/Users/chieme

,underlier,strike,expiration,put/call,open,high,low,close,volume,bid_size,closebid,ask_size,closeask,midpoint,weighted_midpoint,optiontick,openinterest,underlier_price,rf_rate,dividend,rf_rate_name,datetime,bs_iv,midpoint_bs_iv,weighted_midpoint_bs_iv,bid_bs_iv,ask_bs_iv,binomial_iv,midpoint_binomial_iv,weighted_midpoint_binomial_iv,bid_binomial_iv,ask_binomial_iv,delta,gamma,vega,theta,rho,vanna,volga,midpoint_delta,midpoint_gamma,midpoint_vega,midpoint_theta,midpoint_rho,midpoint_vanna,midpoint_volga,weighted_midpoint_delta,weighted_midpoint_gamma,weighted_midpoint_vega,weighted_midpoint_theta,weighted_midpoint_rho,weighted_midpoint_vanna,weighted_midpoint_volga,bid_binomial_delta,bid_binomial_gamma,bid_binomial_vega,bid_binomial_theta,bid_binomial_rho,bid_binomial_vanna,bid_binomial_volga,ask_binomial_delta,ask_binomial_gamma,ask_binomial_vega,ask_binomial_theta,ask_binomial_rho,ask_binomial_vanna,ask_binomial_volga,binomial_delta,binomial_gamma,binomial_vega,binomial_theta,binomial_rho,binomial_vanna,binomial_volga,midpoint_binomial_delta,midpoint_binomial_gamma,midpoint_binomial_vega,midpoint_binomial_theta,midpoint_binomial_rho,midpoint_binomial_vanna,midpoint_binomial_volga,dollar_delta,midpoint_dollar_delta,weighted_midpoint_dollar_delta,last_updated,midpoint_bs_vol_resolve,midpoint_binomial_vol_resolve
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-09-18,AAPL,20.0,2025-03-21,C,0.00,0.00,0.00,0.00,0,55,199.20,53,202.75,200.975,200.94213,AAPL20250321C20,0,220.690002,0.04735,0.004521,^IRX,2024-09-18,0.000000,1.553734,1.529898,0.0,2.188638,0.00000,5.000000,5.000000,0.0001,0.568237,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.994736,0.000037,0.014283,-0.005695,0.093465,-1.053431,453.472093,0.994963,0.000035,0.013302,-0.005206,0.093883,-1.097849,477.023705,0.997725,2.842171e-10,0.0,0.000194,0.098378,-3.405488e+08,1.820725e+15,0.997725,5.684342e-10,2.826610e-09,0.000194,0.098378,-10.192711,9912.097966,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.990737,2.483773e-05,0.030470,-0.039280,0.013524,2.163292e-01,-8.325827e+01,0.000000,219.528278,219.578330,2025-04-24 00:11:36,0,0
2024-09-19,AAPL,20.0,2025-03-21,C,0.00,0.00,0.00,0.00,0,50,207.45,50,210.65,209.050,209.05000,AAPL20250321C20,0,228.869995,0.04615,0.004441,^IRX,2024-09-19,0.000000,1.508929,1.508929,0.0,2.147256,0.00000,5.000000,5.000000,0.0001,0.582920,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.995496,0.000029,0.011571,-0.004353,0.094139,-1.166161,535.717134,0.995496,0.000029,0.011571,-0.004353,0.094139,-1.166161,535.717134,0.997778,0.000000e+00,0.0,0.000307,0.097915,-3.465306e+08,1.949704e+15,0.997778,-5.684342e-10,3.822137e-09,0.000307,0.097915,-9.845092,9831.549992,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.990927,2.359883e-05,0.030967,-0.040139,0.013825,2.130445e-01,-8.504779e+01,0.000000,227.839233,227.839233,2025-04-24 00:11:36,0,0
2024-09-20,AAPL,20.0,2025-03-21,C,0.00,0.00,0.00,0.00,0,60,206.40,30,210.60,208.500,207.80000,AAPL20250321C20,0,228.199997,0.04545,0.004282,^IRX,2024-09-20,0.000000,1.592857,0.000000,0.0,2.300400,0.00000,0.000100,0.000100,0.0001,0.812651,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.994855,0.000036,0.014795,-0.006093,0.092313,-1.012564,447.833334,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.997869,-2.842171e-10,0.0,0.000237,0.097427,-3.470443e+08,1.944253e+15,0.997866,9.038104e-08,1.914199e-05,0.000233,0.097423,-4.902838,3606.482996,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.997869,-2.842171e-10,0.000000,0.000237,0.097427,-3.470443e+08,1.944253e+15,0.000000,227.025929,0.000000,2025-04-24 00:11:36,0,0
2024-09-23,AAPL,20.0,2025-03-21,C,0.00,0.00,0.00,0.00,0,50,205.45,30,208.10,206.775,206.44375,AAPL20250321C20,0,226.470001,0.04545,0.004294,^IRX,2024-09-23,0.000000,1.603729,1.097023,0.0,2.119952,0.00000,0.000100,0.000100,0.0001,0.655600,0.000000,0.

In [26]:
pd.options.display.max_columns = None
# request[request.weighted_midpoint.isna()]
data = request.spot_data.copy()
data

,open,high,low,close,volume,bid_size,closebid,ask_size,closeask,midpoint,weighted_midpoint,openinterest,strike,expiration,put/call,optiontick,underlier,underlier_price,rf_rate,dividend,rf_rate_name,datetime,bs_iv,midpoint_bs_iv,weighted_midpoint_bs_iv,bid_bs_iv,ask_bs_iv,binomial_iv,midpoint_binomial_iv,weighted_midpoint_binomial_iv,bid_binomial_iv,ask_binomial_iv
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-12-04,0.0,0.00,0.0,0.0,0,88,159.05,34,160.70,159.875,159.509836,3,570.0,2026-01-16 00:00:00,P,NVDA20260116P570,NVDA,45.509998,0.05215,0.000257,^IRX,2023-12-04,0.0,0.0,0.0,0.0,0.0,0.0000,0.0001,0.0001,0.0001,0.0001
2023-12-05,0.0,0.00,0.0,0.0,0,55,154.45,37,156.30,155.375,155.194022,3,570.0,2026-01-16 00:00:00,P,NVDA20260116P570,NVDA,46.566002,0.05223,0.000264,^IRX,2023-12-05,0.0,0.0,0.0,0.0,0.0,0.0000,0.0001,0.0001,0.0001,0.0001
2023-12-06,0.0,0.00,0.0,0.0,0,47,159.50,37,161.15,160.325,160.226786,3,570.0,2026-01-16 00:00:00,P,NVDA20260116P570,NVDA,45.502998,0.05243,0.000344,^IRX,2023-12-06,0.0,0.0,0.0,0.0,0.0,0.0000,0.0001,0.0001,0.0001,0.0001
2023-12-07,0.0,0.00,0.0,0.0,0,22,154.15,51,155.90,155.025,155.372603,3,570.0,2026-01-16 00:00:00,P,NVDA20260116P570,NVDA,46.596001,0.05243,0.000352,^IRX,2023-12-07,0.0,0.0,0.0,0.0,0.0,0.0000,0.0001,0.0001,0.0001,0.0001
2023-12-08,0.0,0.00,0.0,0.0,0,51,148.75,42,150.40,149.575,149.495161,3,570.0,2026-01-16 00:00:00,P,NVDA20260116P570,NVDA,47.506001,0.05233,0.000343,^IRX,2023-12-08,0.0,0.0,0.0,0.0,0.0,0.0000,0.0001,0.0001,0.0001,0.0001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-04,26.5,26.65,26.0,26.0,8,85,24.95,99,25.90,25.425,25.461141,103,570.0,2026-01-16 00:00:00,P,NVDA20260116P570,NVDA,116.436996,0.05243,0.000139,^IRX,2024-06-04,0.0,0.0,0.0,0.0,0.0,0.0001,0.0001,0.0001,0.0001,0.0001
2024-06-05,0.0,0.00,0.0,0.0,0,91,22.85,100,23.80,23.325,23.347382,109,570.0,2026-01-16 00:00:00,P,NVDA20260116P570,NVDA,122.440002,0.05240,0.000137,^IRX,2024-06-05,0.0,0.0,0.0,0.0,0.0,0.0000,0.0001,0.0001,0.0001,0.0001
2024-06-06,0.0,0.00,0.0,0.0,0,77,23.50,80,24.60,24.050,24.060510,109,570.0,2026-01-16 00:00:00,P,NVDA20260116P570,NVDA,120.998001,0.05240,0.000131,^IRX,2024-06-06,0.0,0.0,0.0,0.0,0.0,0.0000,0.0001,0.0001,0.0001,0.0001
